Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [56]:
df['catwasher'] = df['cats_allowed'] + df['dishwasher']

#feature shows that cats are allowed and apartment has a dishwasher

df['catwasher'] = df['catwasher'].replace(to_replace=1,value=0)

df['catwasher'] = df['catwasher'].replace(to_replace=2,value=1)

#standardize so apartments with both are 1, apartments with neither or only one are 0

df['catwasher'].value_counts()

0    39701
1     9116
Name: catwasher, dtype: int64

In [57]:
df['oldtimey'] = df['doorman'] + df['pre-war']

#feature shows apartment has a doorman and was built pre-war

df['oldtimey'] = df['oldtimey'].replace(to_replace=1,value=0)

df['oldtimey'] = df['oldtimey'].replace(to_replace=2,value=1)

#standardize so apartments with both are 1, apartments with neither or only one are 0

df['oldtimey'].value_counts()

0    45479
1     3338
Name: oldtimey, dtype: int64

In [0]:
train = df[(df['created'] > '2016-04-01 00:00:00') & (df['created'] < '2016-06-01 00:00:00')]
test = df[(df['created'] > '2016-06-01 00:00:00') & (df['created'] < '2016-07-01 00:00:00')]

#split into train and test sets

In [0]:
target = 'price'

features = ['latitude', 'catwasher']

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()

model.fit(X_train, y_train)

#fit model

y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: ${mae:.2f} ')

#not sure why MAE works here and not below

In [0]:
model.intercept_, model.coef_

#gets intercept and coefficients

In [0]:
print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

#gets intercept and coefficients but readable

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

msetest = mean_squared_error(y_test, y_pred)

rmsetest = np.sqrt(msetest)

maetest = mean_absolute_error(y_test, y_pred)

r2test = r2_score(y_test, y_pred)

print('Root Mean Squared Error Test:', rmsetest)
print('Mean Absolute Error Test:', maetest)
print('R^2:', r2test)

#regression metrics for test data

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

msetrain = mean_squared_error(y_train, y_pred)

rmsetrain = np.sqrt(msetrain)

maetrain = mean_absolute_error(y_train, y_pred)

r2train = r2_score(y_train, y_pred)

print('Root Mean Squared Error Train:', rmsetrain)
print('Mean Absolute Error Train:', maetrain)
print('R^2:', r2train)

#regression metrics for training data not working out for some reason